# Partie I: design d’ETL
L’objectif de cette partie est de produire des spécifications d’ETL. A partir de données sources et à l’aide d’outils de la communautés OHDSI, vous produirez un document technique permettant d’expliciter les transformations nécessaires pour adapter les données sources au format OMOP CDM v5.4. Ce document doit permettre à tout intégrateur d’implémenter un ETL de ces données sources sans ambigüité ni interprétations personnelles.

## Études de données synthétiques
Données téléchargées depuis https://synthea.mitre.org/downloads

### Analyse du rapport généré par WhiteRabbit

In [34]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl') # Ignorer le warning de manque de formatage du fichier excel

path = "ReportFrequency.xlsx" # Path vers le rapport
scan_report=pd.ExcelFile(path)
print("Le rapport contient " + str(len(scan_report.sheet_names)) + " feuilles excel :")
print(scan_report.sheet_names)

Le rapport contient 19 feuilles excel :
['Field Overview', 'Table Overview', 'conditions.csv', 'providers.csv', 'patients.csv', 'allergies.csv', 'observations.csv', 'devices.csv', 'procedures.csv', 'supplies.csv', 'payers.csv', 'careplans.csv', 'payer_transitions.csv', 'immunizations.csv', 'imaging_studies.csv', 'medications.csv', 'encounters.csv', 'organizations.csv', '_']


#### Field Overview

In [3]:
scan_report.parse('Field Overview').head(5)

,Table,Field,Description,Type,Max length,N rows,N rows checked,Fraction empty,N unique values,Fraction unique,Average,Standard Deviation,Min,25%,Median,75%,Max
0,conditions.csv,START,NaN,DATE,10.0,-1.0,8376.0,0.00000,5356.0,0.639446,2003-06-28,6.646630e+03,1916-12-28,1995-02-19,2011-08-26,2016-01-12,2020-04-25
1,conditions.csv,STOP,NaN,DATE,10.0,-1.0,8376.0,0.45499,2924.0,0.349093,2012-03-13,3.839641e+03,1917-09-06,2011-10-29,2014-09-17,2017-06-15,2020-04-27
2,conditions.csv,PATIENT,NaN,VARCHAR,36.0,-1.0,8376.0,0.00000,1152.0,0.137536,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conditions.csv,ENCOUNTER,NaN,VARCHAR,36.0,-1.0,8376.0,0.00000,7650.0,0.913324,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,conditions.csv,CODE,NaN,INT,15.0,-1.0,8376.0,0.00000,129.0,0.015401,1949885905490.615234,2.219533e+13,1734006.0,53741008.0,127013003.0,283371005.0,368581000119106.0


La table résume les champs présentés dans les autres feuilles.

On retrouve bien les 16 noms de fichiers csv (plus la ligne "NaN" qui modélise la ligne vide entre les différentes tables). Pour chacun des fichiers, on a une ligne par colonne présente dans le csv associé.
Les colonnes de la table nous donnent des informations statistiques sur les différentes colonnes des différents fichiers. Les colonnes utilisant des valeurs numériques (y compris les dates) présentent en plus des informations statistiques sur les valeurs qui les composent.
On a également accès aux proportions de valeurs uniques et/ou vides, et au nombre de cellules lues pour la génération du rapport.

Les jeux de données contiennent des informations variées et de sources multiples. On note la présence de certains champs dans plusieurs feuilles différentes, tels les champs:
- "PATIENT"
- "ENCOUNTER"
- "CODE"
- "DESCRIPTION"
- "REASONCODE"
- "REASONDESCRIPTION"
- "START_YEAR"
- "END_YEAR"
- ...

Il conviendra de bien faire attention à relier tous les champs dans les bonnes tables.

Nous pouvons également trouver le pourcentage de lignes vides dans chaque colonne. 36 colonnes comportent des lignes vides. Si on retire la feuille devices, on obtient ainsi 30 colonnes incomplètes. Il faudra s'assurer que ces champs ne sont pas des champs obligatoires pour une ou des tables OMOP.

In [33]:
scan_report.parse("Field Overview")[scan_report.parse("Field Overview")["Fraction empty"] != 0]

,Table,Field,Description,Type,Max length,N rows,N rows checked,Fraction empty,N unique values,Fraction unique,Average,Standard Deviation,Min,25%,Median,75%,Max
1,conditions.csv,STOP,NaN,DATE,10.0,-1.0,8376.0,0.454990,2924.0,0.349093,2012-03-13,3.839641e+03,1917-09-06,2011-10-29,2014-09-17,2017-06-15,2020-04-27
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,patients.csv,DEATHDATE,NaN,DATE,10.0,-1.0,1171.0,0.853971,172.0,0.146883,1999-04-23,6.309625e+03,1926-03-05,1988-09-20,2003-02-15,2014-02-26,2020-03-30
24,patients.csv,DRIVERS,NaN,VARCHAR,9.0,-1.0,1171.0,0.181896,953.0,0.813834,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,patients.csv,PASSPORT,NaN,VARCHAR,10.0,-1.0,1171.0,0.233134,899.0,0.767720,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,patients.csv,PREFIX,NaN,VARCHAR,4.0,-1.0,1171.0,0.208369,4.0,0.003416,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,patients.csv,SUFFIX,NaN,VARCHAR,3.0,-1.0,1171.0,0.989752,4.0,0.003416,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,patients.csv,MAIDEN,NaN,VARCHAR,16.0,-1.0,1171.0,0.717336,261.0,0.222886,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,patients.csv,MARITAL,NaN,VARCHAR,1.0,-1.0,1171.0,0.324509,3.0,0.002562,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Table Overview

In [4]:
scan_report.parse('Table Overview').head(5)

,Table,Description,N rows,N rows checked,N Fields,N Fields Empty
0,conditions.csv,NaN,-1.0,8376.0,6.0,0.0
1,providers.csv,NaN,-1.0,5855.0,12.0,0.0
2,patients.csv,NaN,-1.0,1171.0,25.0,0.0
3,allergies.csv,NaN,-1.0,597.0,6.0,0.0
4,observations.csv,NaN,-1.0,299697.0,8.0,0.0


On voit ici pour chaque table le nombre de lignes lues, le nombre de champs existants, et de champs vides.

On remarque déjà qu'une feuille est totalement vide (supplies.csv), nous n'avons donc pas à la traiter pour intégrer ces données. Une autre feuille, devices, contient un champ vide. Il faut vérifier si le champ est un champ obligatoire d'une table OMOP, afin d'éviter toute erreur.

In [5]:
scan_report.parse('Table Overview')[scan_report.parse('Table Overview')["N Fields Empty"] != 0]

,Table,Description,N rows,N rows checked,N Fields,N Fields Empty
5,devices.csv,NaN,-1.0,78.0,7.0,1.0
7,supplies.csv,NaN,-1.0,0.0,6.0,6.0


In [6]:
scan_report.parse('Table Overview').sort_values(by=["N rows checked"], ascending=False)

,Table,Description,N rows,N rows checked,N Fields,N Fields Empty
4,observations.csv,NaN,-1.0,299697.0,8.0,0.0
14,encounters.csv,NaN,-1.0,53346.0,15.0,0.0
13,medications.csv,NaN,-1.0,42989.0,13.0,0.0
6,procedures.csv,NaN,-1.0,34981.0,8.0,0.0
11,immunizations.csv,NaN,-1.0,15478.0,6.0,0.0
0,conditions.csv,NaN,-1.0,8376.0,6.0,0.0
1,providers.csv,NaN,-1.0,5855.0,12.0,0.0
10,payer_transitions.csv,NaN,-1.0,3801.0,5.0,0.0
9,careplans.csv,NaN,-1.0,3483.0,9.0,0.0
2,patients.csv,NaN,-1.0,1171.0,25.0,0.0


On note que la table "Observation" est la plus remplie, contenant 6 fois plus de lignes que la table "encounters" qui la suit, ce qui veut certainement dire qu'une encounter amène généralement à plus d'une observation. On peut confirmer cela en regardant dans la feuille "Table Overview", le pourcentage de rencontres uniques au sein des observations est de 6,7%.

#### Condition

In [7]:
scan_report.parse("conditions.csv").head(5)

,START,Frequency,STOP,Frequency.1,PATIENT,Frequency.2,ENCOUNTER,Frequency.3,CODE,Frequency.4,DESCRIPTION,Frequency.5
0,2017-03-21,8.0,NaN,3811.0,6ec18ddf-e9ee-421a-9033-456f558c7b4b,25.0,78880c30-e6ce-4598-a08b-d222b55c64af,7.0,444814009.0,1248.0,Viral sinusitis (disorder),1248.0
1,2016-10-08,8.0,2011-01-18,8.0,9394cb52-7d92-4577-976e-d017af06ed8f,23.0,b27ef5b9-4549-457c-8ef5-f8091d380f7d,5.0,195662009.0,653.0,Acute viral pharyngitis (disorder),653.0
2,1990-02-05,8.0,2012-08-03,6.0,3f336702-bf73-4fc8-bd59-3ba77fd65d0d,23.0,6ba161b6-121f-44e9-9899-6505efe0aea6,5.0,10509002.0,563.0,Acute bronchitis (disorder),563.0
3,2012-03-12,7.0,2017-04-23,6.0,5c06120a-9af5-4204-951b-7a8bfc465df3,22.0,ee5898a0-7865-442a-a004-56db5d81a67e,5.0,72892002.0,516.0,Normal pregnancy,516.0
4,2017-06-03,7.0,2015-01-27,6.0,2c71dd97-7085-416a-aa07-d675bbe3adf2,21.0,d9166bb6-91e8-4e1a-9faa-613b2c1dcfb5,4.0,162864005.0,449.0,Body mass index 30+ - obesity (finding),449.0


La table présente les dates de débuts et de fin d'un état observé pour un patient donné, avec le code de l'état et sa description associée.
L'état le plus observé est la sinusite virale, qui représente presque 15% des diagnostics de rencontres

#### Providers

In [8]:
scan_report.parse("providers.csv").head(5)

,Id,Frequency,ORGANIZATION,Frequency.1,NAME,Frequency.2,GENDER,Frequency.3,SPECIALITY,Frequency.4,...,STATE,Frequency.7,ZIP,Frequency.8,LAT,Frequency.9,LON,Frequency.10,UTILIZATION,Frequency.11
0,74656ce3-3c5a-369c-b15d-69e1d0f94995,1.0,7e79cf8f-66ff-3696-9f73-172be755ac72,403.0,Vern731 Powlowski563,4.0,F,3062.0,GENERAL PRACTICE,1120.0,...,MA,5855.0,02144-2705,404.0,42.390445,453.0,-71.101033,453.0,0.0,4735.0
1,293f20d7-5c01-3e04-bf8d-c65f0862440e,1.0,4215bb3e-96a5-326d-96c1-b54744a390fd,356.0,Leonor133 Nájera755,4.0,M,2793.0,INTERNAL MEDICINE,608.0,...,NaN,NaN,01960-2901,356.0,42.534279,381.0,-70.969782,381.0,28.0,68.0
2,ea69cf24-c6f5-389d-9fb6-93cd21403cdf,1.0,01cf784f-f213-3ab8-bca3-919795beaaf6,329.0,Colby655 Schowalter414,4.0,NaN,NaN,NURSE PRACTITIONER,511.0,...,NaN,NaN,01830-6738,329.0,42.782945,342.0,-71.087436,342.0,30.0,33.0
3,fec9ef0e-dc7f-3062-a19d-f7f1722afdac,1.0,a441cbca-6d65-33ff-980c-d8e58235b7e5,136.0,Mari763 Reynolds644,4.0,NaN,NaN,CLINICAL SOCIAL WORKER,367.0,...,NaN,NaN,02726-1220,136.0,42.376043,272.0,-71.118680,272.0,31.0,31.0
4,fe83d7d5-26ff-3fd1-8851-fe2d88fa5d33,1.0,819cb4e2-fb1c-3917-80c9-fdb705d2cd19,117.0,Lynwood354 Monahan736,4.0,NaN,NaN,PHYSICIAN ASSISTANT,311.0,...,NaN,NaN,02719-5208,123.0,41.738479,151.0,-71.165065,151.0,29.0,25.0


Nous retrouvons ici la liste des praticiens, ainsi que leurs spécialités et localisation.
Nous notons entre autres :
- Les chiffres dans le nom nous paraissent étranges
- Nous n'avons pas trouvé de signification au champ utilisation
- La colonne "STATE" est toujours égale à "MA"

#### Patients

In [9]:
scan_report.parse("patients.csv").head(5)

,Id,Frequency,BIRTHDATE,Frequency.1,DEATHDATE,Frequency.2,SSN,Frequency.3,DRIVERS,Frequency.4,...,ZIP,Frequency.20,LAT,Frequency.21,LON,Frequency.22,HEALTHCARE_EXPENSES,Frequency.23,HEALTHCARE_COVERAGE,Frequency.24
0,87f05059-de42-4630-a35b-edb53d880640,1.0,1942-05-23,10.0,NaN,1000.0,999-56-8425,2.0,NaN,213.0,...,NaN,543.0,42.581626,1.0,-70.967142,1.0,475343.76,1.0,0.00,28.0
1,d78faf3a-dfad-4b6b-a353-e688cd2b8568,1.0,1913-05-30,8.0,1991-02-17,1.0,999-81-2423,1.0,S99976102,2.0,...,2472.0,12.0,42.474901,1.0,-69.968061,1.0,1660466.31,1.0,645.80,8.0
2,4217ed61-af8b-47be-80d2-c84336bc01a2,1.0,1914-09-05,8.0,1958-01-03,1.0,999-75-7372,1.0,S99951275,2.0,...,2151.0,11.0,42.477650,1.0,-71.160521,1.0,26070.08,1.0,516.64,6.0
3,0e1ad739-81d1-4170-841d-414ddc97c93a,1.0,1923-05-15,7.0,2003-02-15,1.0,999-70-6525,1.0,S99958686,2.0,...,2140.0,10.0,42.215817,1.0,-71.551489,1.0,300475.37,1.0,1033.28,5.0
4,bfb101aa-8d36-422b-8e5c-b7275d279de0,1.0,1925-04-22,7.0,2018-05-29,1.0,999-69-3267,1.0,S99973225,2.0,...,2155.0,10.0,42.331358,1.0,-71.167223,1.0,319572.37,1.0,1162.44,4.0


La table regroupe les informations administratives des patients (dont noms, permis de conduire, identifiant unique, assurance maladie, ethnie, localisation ...).

On note qu'un SSN est dupliqué (2 apparitions) alors qu'aucun ID n'est dupliqué pour matcher ce SSN.
Les données semblent toutes provenir du Massachusets

#### Allergies

In [10]:
scan_report.parse("allergies.csv").head(5)

,START,Frequency,STOP,Frequency.1,PATIENT,Frequency.2,ENCOUNTER,Frequency.3,CODE,Frequency.4,DESCRIPTION,Frequency.5
0,2014-04-24,10.0,NaN,533.0,1c2aa038-9366-4c7d-9a3e-52cb753a670f,10.0,d47a0669-53b1-4914-b18d-dd593f7e1767,10.0,419474003.0,81.0,Allergy to mould,81.0
1,2010-04-03,10.0,2017-01-31,5.0,bf35e4fa-ea4f-40a4-8fe6-1f2f26e0aa45,10.0,c90b2536-b388-479c-aa7e-3406fe4c2211,10.0,232347008.0,77.0,Dander (animal) allergy,77.0
2,2002-01-24,10.0,2015-11-30,5.0,e112cedd-a98e-489e-abb0-875420d40397,10.0,a61f97fa-70c3-4366-90e1-7c6fdcba5cbb,10.0,419263009.0,63.0,Allergy to tree pollen,63.0
3,1963-07-23,10.0,1993-07-10,4.0,2174522f-1d23-47cf-b56c-4ce3193c5bab,10.0,89685e6f-cb99-47aa-8304-e9bc4506e7cd,10.0,232350006.0,61.0,House dust mite allergy,61.0
4,2016-04-25,9.0,2013-06-25,4.0,b37c9b35-6fea-4570-b7f6-379baf4c9399,9.0,469fbd8a-ec48-4da9-9165-027144ccf9a0,9.0,418689008.0,60.0,Allergy to grass pollen,60.0


On retrouve ici les informations des rencontres en lien avec les allergies.

On note que seulement 64 patients ont une date de fin pour leur allergie, sur les 597 diagnostiqués.

#### Observations

In [11]:
scan_report.parse("observations.csv").head(5)

,DATE,Frequency,PATIENT,Frequency.1,ENCOUNTER,Frequency.2,CODE,Frequency.3,DESCRIPTION,Frequency.4,VALUE,Frequency.5,UNITS,Frequency.6,TYPE,Frequency.7
0,1998-08-02T13:25:58Z,94.0,cae10920-f977-48b4-a0d3-4d70ad561fd1,9023.0,NaN,30363.0,72514-3,16820.0,Pain severity - 0-10 verbal numeric rating [Sc...,16820.0,1.0,11382.0,mg/dL,51907.0,numeric,278488.0
1,2011-10-27T12:08:54Z,86.0,19d2cfb8-439b-454a-b47e-5274c219005b,8560.0,44b9670e-fd47-4f04-b75d-3346f2cef9e4,79.0,8480-6,12963.0,Systolic Blood Pressure,12963.0,Never smoker,9319.0,{score},26941.0,text,21209.0
2,1968-11-24T09:30:42Z,83.0,3f336702-bf73-4fc8-bd59-3ba77fd65d0d,7322.0,8a3a36ec-980e-449e-b21c-2b73c8980cd3,78.0,8462-4,12963.0,Diastolic Blood Pressure,12963.0,0.0,6144.0,mmol/L,26060.0,NaN,NaN
3,2001-08-19T13:25:58Z,83.0,59cf17d9-6c13-4333-a1cb-cc5fdf63366d,2988.0,95dc9058-ff5e-41af-bb0d-e573df1c389c,73.0,29463-7,12552.0,Tobacco smoking status NHIS,12552.0,3.0,4752.0,mm[Hg],25926.0,NaN,NaN
4,1971-10-10T09:30:42Z,83.0,941f66eb-5ca6-45cc-b205-6de1f1a59233,2916.0,dad811f9-6f39-434e-9ace-cb4341f0a4f2,68.0,9279-1,12552.0,Body Height,12552.0,2.0,4112.0,/min,25104.0,NaN,NaN


La table regroupe toutes les observations et leurs informations.

On note que 10% des observations sont sur le niveau de douleur, le premier test passé en entrant à l'hôpital

On remarque que les données sont majoritairement numériques, et que parmi les données non numériques, la majorité est réponse au statut fumeur. Les réponses à cette question (57 fumeurs quotidiens sur 12552 patients interrogés) laissent penser que les patients cachent la vérité. Les données étant tirées des États-Unis, on supposera que la peur d'un acte non couvert car lié à la cigarette pousse les patients à cacher la vérité

#### Devices

In [12]:
scan_report.parse("devices.csv").head(5)

,START,Frequency,STOP,Frequency.1,PATIENT,Frequency.2,ENCOUNTER,Frequency.3,CODE,Frequency.4,DESCRIPTION,Frequency.5,UDI,Frequency.6
0,1973-07-19T04:25:46Z,1.0,NaN,78.0,fd1e9a6d-5b50-4a6f-bd61-c884ab89f5c6,2.0,e7d4e283-2a2c-4733-9456-0e7ad7793114,1.0,72506001.0,47.0,Implantable defibrillator device (physical ob...,47.0,(01)30949086444789(11)470718(17)720801(10)1045...,1.0
1,1988-05-17T13:04:22Z,1.0,NaN,NaN,668ad49e-46d7-45ec-a4b4-dab0961d180b,2.0,ef913b44-a6af-4a36-9085-75e0cf514497,1.0,705643001.0,21.0,Coronary artery stent (physical object),21.0,(01)84512353376760(11)620911(17)870926(10)4832...,1.0
2,1965-07-11T19:05:21Z,1.0,NaN,NaN,2c71dd97-7085-416a-aa07-d675bbe3adf2,2.0,b7dd8a72-8ec2-482a-bc99-d043362e08d6,1.0,706004007.0,10.0,Implantable cardiac pacemaker (physical object),10.0,(01)11821950664484(11)131010(17)381025(10)7096...,1.0
3,2007-09-10T04:16:32Z,1.0,NaN,NaN,85a0da4b-6e8d-492c-976b-748e39daf4ef,2.0,4f1264b0-40f7-4e5f-af2c-f3dc83345dc2,1.0,NaN,NaN,NaN,NaN,(01)87969465031282(11)111028(17)361111(10)7964...,1.0
4,1991-02-18T01:51:56Z,1.0,NaN,NaN,87f05059-de42-4630-a35b-edb53d880640,1.0,1785e2e2-41cd-4e80-bda9-42ef955f47c8,1.0,NaN,NaN,NaN,NaN,(01)83989236019812(11)870614(17)120628(10)1790...,1.0


Cette table liste les différents appareils implantés à des patients au cours de rencontres.

Seuls trois types de devices sont répertoriés, dont plus de la moitié étant des défibrillateurs. Il n'y a pas de date d'arrêt bien que certaines opérations datent d'il y a des décennies, on suppose que les remplacements d'appareil ne sont pas comptabilisés comme 2 implants différents.

#### Procedures

In [13]:
scan_report.parse('procedures.csv').head(5)

,DATE,Frequency,PATIENT,Frequency.1,ENCOUNTER,Frequency.2,CODE,Frequency.3,DESCRIPTION,Frequency.4,BASE_COST,Frequency.5,REASONCODE,Frequency.6,REASONDESCRIPTION,Frequency.7
0,2016-12-30T13:51:47Z,24.0,cae10920-f977-48b4-a0d3-4d70ad561fd1,1273.0,0f2eccff-7741-4ff2-a13a-656cbd78394a,21.0,430193006.0,5632.0,Medication Reconciliation (procedure),5632.0,516.65,8031.0,NaN,15544.0,NaN,15544.0
1,2012-04-06T12:19:58Z,24.0,19d2cfb8-439b-454a-b47e-5274c219005b,1251.0,0f0c75b5-7aa2-4e50-9bce-35ec36397bcf,20.0,265764009.0,3389.0,Renal dialysis (procedure),3389.0,621.45,3.0,72892002.0,14578.0,Normal pregnancy,14578.0
2,2016-12-17T18:46:04Z,24.0,3f336702-bf73-4fc8-bd59-3ba77fd65d0d,1232.0,52a69be5-95a4-417f-9e44-4e6ca77e47a5,20.0,274804006.0,2705.0,Auscultation of the fetal heart,2705.0,2280.21,3.0,49436004.0,1231.0,Atrial Fibrillation,1231.0
3,2012-12-22T06:41:05Z,24.0,3acf9313-1874-4dff-ab2a-3187516d92d6,478.0,1a98a614-f17d-4714-ad15-975b91e6f84d,20.0,225158009.0,2705.0,Evaluation of uterine fundal height,2705.0,3375.82,3.0,254837009.0,653.0,Malignant neoplasm of breast (disorder),653.0
4,2018-03-29T21:30:58Z,24.0,10a4a009-2d24-4512-8506-f8d8c36255d5,317.0,43a43bbe-d73d-44be-8e2d-5b8e9a842f19,20.0,180256009.0,1497.0,Subcutaneous immunotherapy,1497.0,498.51,3.0,424132000.0,555.0,Non-small cell carcinoma of lung TNM stage 1 ...,555.0


On retrouve ici toutes les procédures médicales réalisées, ainsi que leurs raisons, dates, patients et praticien.
On remarque que les procédures sont sans motif renseigné ou liées à la grossesse, qui représentent à eux seuls 86% des procédures.
Le plus grand groupe de patients, 23% d'entre eux, paie la même somme : 516,65 euros.
De plus, 3 patients ont à eux seuls presque 11% des traitements.

#### Supplies

In [14]:
scan_report.parse("supplies.csv").head(5)

,DATE,Frequency,PATIENT,Frequency.1,ENCOUNTER,Frequency.2,CODE,Frequency.3,DESCRIPTION,Frequency.4,QUANTITY,Frequency.5


La table est vide.

#### Payers

In [15]:
scan_report.parse("payers.csv").head(5)

,Id,Frequency,NAME,Frequency.1,ADDRESS,Frequency.2,CITY,Frequency.3,STATE_HEADQUARTERED,Frequency.4,...,COVERED_IMMUNIZATIONS,Frequency.16,UNCOVERED_IMMUNIZATIONS,Frequency.17,UNIQUE_CUSTOMERS,Frequency.18,QOLS_AVG,Frequency.19,MEMBER_MONTHS,Frequency.20
0,b3221cfc-24fb-339e-823d-bc4136cbc4ed,1.0,Dual Eligible,1.0,7500 Security Blvd,3.0,Baltimore,3.0,MD,3.0,...,7022,1.0,0.0,4.0,198,1.0,0.932301,1.0,74040,1.0
1,d47b3510-2895-3b70-9897-342d681c769d,1.0,Aetna,1.0,NaN,1.0,NaN,1.0,CT,2.0,...,0,1.0,156.0,1.0,297,1.0,1.134518,1.0,29760,1.0
2,047f6ec3-6215-35eb-9608-f9dda363a44c,1.0,Medicare,1.0,Michigan Plaza,1.0,Bloomfield,1.0,NaN,1.0,...,2557,1.0,160.0,1.0,99,1.0,0.927714,1.0,3348,1.0
3,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,1.0,Humana,1.0,220 Virginia Ave,1.0,Chicago,1.0,MN,1.0,...,6661,1.0,3644.0,1.0,25,1.0,0.931283,1.0,71304,1.0
4,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,1.0,Anthem,1.0,900 Cottage Grove Rd,1.0,Minnetonka,1.0,IL,1.0,...,223,1.0,195.0,1.0,307,1.0,0.812137,1.0,84276,1.0


La table contient toutes les informations des payants.

On remarque qu'une entreprise n'a rien couvert, et que pour chaque catégorie à couvrir, un acteur a tout couvert alors qu'un autre n'a rien couvert

#### Careplans

In [16]:
scan_report.parse("careplans.csv").head(5)

,Id,Frequency,START,Frequency.1,STOP,Frequency.2,PATIENT,Frequency.3,ENCOUNTER,Frequency.4,CODE,Frequency.5,DESCRIPTION,Frequency.6,REASONCODE,Frequency.7,REASONDESCRIPTION,Frequency.8
0,d7482540-24ef-472e-9d75-a82e153b1b96,1.0,2014-11-12,5.0,NaN,1532.0,66135cde-428a-4cf7-bc1d-d5ddce40d9cd,12.0,fa9c2111-5159-4d62-8038-b5513af3a337,2.0,53950000.0,633.0,Respiratory therapy,633.0,10509002.0,633.0,Acute bronchitis (disorder),633.0
1,287c943a-8e90-4e6b-be43-10e9dacf9559,1.0,2017-06-01,5.0,2013-01-20,4.0,4f9dcdaa-fb47-412b-8288-03f8cbbc321e,12.0,bac29aef-34af-4807-8006-cec4c29f288f,2.0,134435003.0,397.0,Routine antenatal care,397.0,72892002.0,397.0,Normal pregnancy,397.0
2,ddc345b1-c0ef-458c-908f-24266287fa3a,1.0,2011-11-13,5.0,2011-01-18,4.0,3acf9313-1874-4dff-ab2a-3187516d92d6,10.0,3f03fdf7-4c41-462f-b840-a81cfd4143ea,2.0,698360004.0,339.0,Diabetes self management plan,339.0,NaN,327.0,NaN,327.0
3,d0747c3d-756c-4745-b9b3-eda8c1dbd189,1.0,2013-10-04,5.0,2014-01-10,4.0,df7c1d66-eac2-49bd-9d12-ee17e8758f68,9.0,26ade70b-82b3-455b-b283-188c9b281811,2.0,443402002.0,302.0,Lifestyle education regarding hypertension,302.0,59621000.0,302.0,Hypertension,302.0
4,0d46ddee-0703-47f1-8e2b-9ae3ba00a192,1.0,1941-07-08,4.0,2019-05-12,4.0,6dafa93d-3698-4124-9bbd-7429cc2a7374,9.0,b430fc7d-ffcc-4f5c-83ad-6e7b36681003,2.0,91251008.0,230.0,Physical therapy procedure,230.0,15777000.0,295.0,Prediabetes,295.0


Le fichier présente les traitements suivis ou ayant été suivis par les patients.
44% des traitements n'ont pas encore de fin.
327 traitements n'ont par ailleurs pas de code de raison, ni de description de raison associée. Quel genre de traitement cela peut-il être ?

#### Payer Transitions

In [17]:
scan_report.parse("payer_transitions.csv").head(5)

,PATIENT,Frequency,START_YEAR,Frequency.1,END_YEAR,Frequency.2,PAYER,Frequency.3,OWNERSHIP,Frequency.4
0,baffdce5-d444-4c8c-a815-e7d00a01f7ca,23.0,1992.0,85.0,1998.0,82.0,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,1416.0,Self,2299.0
1,e974e5c3-9b22-41f2-b3a3-c12848f29a73,21.0,1988.0,77.0,1995.0,80.0,6e2f1a2d-27bd-3701-8d08-dae202c58632,352.0,Guardian,779.0
2,23365743-e8f2-4aba-8242-15463f81f81b,19.0,1999.0,77.0,2015.0,77.0,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,340.0,Spouse,487.0
3,0325261f-61eb-46f8-acc6-89d15053fecd,18.0,1986.0,73.0,1991.0,76.0,047f6ec3-6215-35eb-9608-f9dda363a44c,318.0,NaN,236.0
4,3a645748-8ddc-4876-8674-5c1ac5bf2899,18.0,1981.0,68.0,2018.0,76.0,d47b3510-2895-3b70-9897-342d681c769d,307.0,NaN,NaN


Le fichier décrit les informations de responsabilité de paiement.
La majorité des paiements sont assumés par le patient ou son épouse.

#### Immunizations

In [18]:
scan_report.parse("immunizations.csv").head(5)

,DATE,Frequency,PATIENT,Frequency.1,ENCOUNTER,Frequency.2,CODE,Frequency.3,DESCRIPTION,Frequency.4,BASE_COST,Frequency.5
0,2019-04-03T13:37:35Z,10.0,4217ed61-af8b-47be-80d2-c84336bc01a2,36.0,47d81e79-eac9-4963-aed3-e693a7a2a273,6.0,140.0,9385.0,Influenza seasonal injectable preservative ...,9385.0,140.52,15478.0
1,2018-10-31T13:37:35Z,10.0,8b18172e-bdf3-4998-aa8b-0db6887cdf53,36.0,a93f0907-db09-4855-a220-641b2fa253a1,6.0,113.0,830.0,Td (adult) preservative free,830.0,NaN,NaN
2,2019-01-02T13:37:35Z,10.0,17c1cce5-2df1-4597-90fa-60cbe557b210,36.0,589d96f3-d65f-4679-8fa6-e0e67ac4f8dc,6.0,133.0,661.0,Pneumococcal conjugate PCV 13,661.0,NaN,NaN
3,2011-12-06T20:46:46Z,7.0,bc95d200-52fc-49cd-b128-8f4150cb4714,36.0,884b05cf-7fd4-4f32-a9ac-815f67ca3d70,6.0,20.0,572.0,DTaP,572.0,NaN,NaN
4,2010-09-04T16:16:46Z,6.0,ee74256b-7850-4a07-9034-6497aee04256,36.0,6484f157-8a46-4eb1-86e5-12f162b71db5,6.0,10.0,481.0,IPV,481.0,NaN,NaN


La feuille regroupe les vaccins injectés aux patients.
Le vaccin le plus utilisé, dans 60% des cas, sont des vaccins contre la grippe saisonnière.
On note que le coût est toujours le même peu importe le type de vaccin.

#### Imaging Studies

In [19]:
scan_report.parse("imaging_studies.csv").head(5)

,Id,Frequency,DATE,Frequency.1,PATIENT,Frequency.2,ENCOUNTER,Frequency.3,BODYSITE_CODE,Frequency.4,BODYSITE_DESCRIPTION,Frequency.5,MODALITY_CODE,Frequency.6,MODALITY_DESCRIPTION,Frequency.7,SOP_CODE,Frequency.8,SOP_DESCRIPTION,Frequency.9
0,501200f8-4cec-4fb6-9b6e-31f2f7553a67,1.0,2020-03-28T12:12:10Z,3.0,941f66eb-5ca6-45cc-b205-6de1f1a59233,59.0,5f486dd5-9d6b-44c8-b70e-03b46d7d8bc4,3.0,51185008.0,484.0,Thoracic structure (body structure),404.0,CT,328.0,Computed Tomography,328.0,1.2.840.10008.5.1.4.1.1.1.1,354.0,CT Image Storage,328.0
1,4221534c-d379-4c6b-a22e-d7eae3fa2609,1.0,2013-02-26T13:04:22Z,3.0,d4adfa2e-89ba-4ae6-9770-ef5a7ec4173a,59.0,3b8e60c5-a48f-4467-9acb-c621571fc7cc,3.0,261179002.0,172.0,thoracic,113.0,DX,278.0,Digital Radiography,278.0,1.2.840.10008.5.1.4.1.1.2,328.0,Digital X-Ray Image Storage,217.0
2,697f923a-d9f8-4e29-988b-b6345f06a16c,1.0,2016-04-15T13:04:22Z,3.0,b2b612b0-d4ac-4b83-8d33-4ed6b295c01c,46.0,464acd51-87e3-4487-96c7-1de14d76bf22,2.0,40983000.0,48.0,Thoracic structure,61.0,US,173.0,Ultrasound,173.0,1.2.840.10008.5.1.4.1.1.3.1,172.0,Ultrasound Multiframe Image Storage,172.0
3,68be9f41-c0f6-427e-8cef-b94ea516c31c,1.0,2019-03-21T23:46:24Z,3.0,e1b30dc8-2618-42c1-a3f8-9c2d8ac49c5f,36.0,2c3283e4-7235-4d65-b5bc-27e99bf12561,2.0,8205005.0,47.0,Thoracic,59.0,CR,76.0,Computed Radiography,76.0,1.2.840.10008.5.1.4.1.1.6.1,1.0,Digital X-Ray Image Storage – for Presentation,137.0
4,9767d744-0269-4635-95fa-4c9ae7e63cbb,1.0,2016-10-05T06:04:10Z,3.0,36421d27-328e-4b13-9fca-903c19fe49dd,36.0,97d800fe-8d91-4f9c-a14c-b70e2669cecd,2.0,344001.0,32.0,Arm,48.0,NaN,NaN,NaN,NaN,NaN,NaN,Ultrasound Image Storage,1.0


Le fichier regroupe toutes les informations liées à une prise d'imagerie sur un patient.
On note que 4 catégories du site imagé réfère à la même notion du thorax, mais utilisant des terminologies différentes, ce qui pourrait fausser les études sur ce champ.
Ce type d'imagerie représente en plus presque 75% des images prises sur des patients, ce qui aggrave l'impact de l'erreur de terminologie.

#### Medications

In [20]:
scan_report.parse("medications.csv").head(5)

,START,Frequency,STOP,Frequency.1,PATIENT,Frequency.2,PAYER,Frequency.3,ENCOUNTER,Frequency.4,...,PAYER_COVERAGE,Frequency.8,DISPENSES,Frequency.9,TOTALCOST,Frequency.10,REASONCODE,Frequency.11,REASONDESCRIPTION,Frequency.12
0,2014-09-22T13:57:11Z,22.0,NaN,1895.0,3f336702-bf73-4fc8-bd59-3ba77fd65d0d,3313.0,7caa7254-5050-3b5e-9eae-bd5ea30e809c,12641.0,2d8c1eef-6b08-4c6e-a594-86569d33c5d6,12.0,...,0.00,25738.0,12.0,17680.0,3161.88,10284.0,59621000.0,11766.0,Hypertension,11766.0
1,2017-01-13T20:51:21Z,20.0,2017-01-13T20:51:21Z,20.0,19d2cfb8-439b-454a-b47e-5274c219005b,1614.0,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,8495.0,15f2338a-d5ab-425f-8878-ba9f51872e66,11.0,...,223.49,4375.0,1.0,16678.0,263.49,5135.0,NaN,11117.0,NaN,11117.0
2,2006-12-08T09:55:43Z,18.0,2006-12-08T09:55:43Z,18.0,2c71dd97-7085-416a-aa07-d675bbe3adf2,1563.0,6e2f1a2d-27bd-3701-8d08-dae202c58632,4882.0,2f09cd6e-8a42-4a82-ae77-a752e1fb7a59,11.0,...,188.49,1628.0,2.0,2139.0,526.98,386.0,44054006.0,7007.0,Diabetes,7007.0
3,2007-02-02T09:55:43Z,18.0,2007-02-02T09:55:43Z,18.0,cae10920-f977-48b4-a0d3-4d70ad561fd1,1551.0,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,4459.0,f00f74ce-349b-44ba-b64e-e7503c6e64e2,10.0,...,203.49,1605.0,3.0,1693.0,790.47,344.0,233678006.0,4144.0,Childhood asthma,4144.0
4,2001-10-09T15:08:21Z,16.0,2014-09-22T13:57:11Z,18.0,3acf9313-1874-4dff-ab2a-3187516d92d6,1237.0,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,3056.0,ba010d7f-1b45-4c85-b246-ee4a3d59038a,10.0,...,163.49,88.0,4.0,693.0,2898.39,185.0,271737000.0,3388.0,Anemia (disorder),3388.0


La feuille contient les informations des prises de médicaments.

La majorité des médicaments délivrés sont des médicaments pour traiter l'hypertension et le diabète. On note également que plus d'un quart des médicaments sont données sans raison particulière.
Environ 60% des médicaments ne sont par ailleurs pas couverts aux États-Unis.

#### Encounters

In [21]:
scan_report.parse("encounters.csv").head(5)

,Id,Frequency,START,Frequency.1,STOP,Frequency.2,PATIENT,Frequency.3,ORGANIZATION,Frequency.4,...,BASE_ENCOUNTER_COST,Frequency.10,TOTAL_CLAIM_COST,Frequency.11,PAYER_COVERAGE,Frequency.12,REASONCODE,Frequency.13,REASONDESCRIPTION,Frequency.14
0,2d495f23-a3dd-4175-8e71-fed82b24d7f7,1.0,1973-07-27T23:35:03Z,7.0,1990-02-06T20:06:53Z,4.0,cae10920-f977-48b4-a0d3-4d70ad561fd1,2006.0,be4c63f3-8d38-3fa9-a183-62045b5c85f6,3217.0,...,129.16,52923.0,129.16,52923.0,0.00,13085.0,NaN,39569.0,NaN,39569.0
1,bac8ad91-863f-43b1-a4fc-79783ef9d9e0,1.0,1990-02-05T20:06:53Z,6.0,1954-02-27T17:57:52Z,4.0,19d2cfb8-439b-454a-b47e-5274c219005b,1606.0,d311e70d-86e7-3c03-b115-53892bcf7ef1,1972.0,...,77.49,423.0,77.49,423.0,129.16,9616.0,72892002.0,3779.0,Normal pregnancy,3779.0
2,e963e35c-3798-4b48-b53e-7e4e74853851,1.0,1986-11-07T19:13:29Z,5.0,1947-01-18T17:42:52Z,4.0,3f336702-bf73-4fc8-bd59-3ba77fd65d0d,1538.0,d692e283-0833-3201-8e55-4f868a9c0736,1658.0,...,NaN,NaN,NaN,NaN,69.16,8921.0,55822004.0,2273.0,Hyperlipidemia,2273.0
3,8366230c-d15b-4f29-af4c-3fbfe4e4194f,1.0,1985-10-04T23:35:03Z,5.0,1980-07-26T17:57:52Z,4.0,3acf9313-1874-4dff-ab2a-3187516d92d6,826.0,e002090d-4e92-300e-b41e-7d1f21dee4c6,1028.0,...,NaN,NaN,NaN,NaN,89.16,7133.0,444814009.0,1378.0,Viral sinusitis (disorder),1378.0
4,0bdf59f3-7572-45c6-b405-f7e89c304757,1.0,2011-11-09T06:09:18Z,5.0,1967-05-13T17:42:52Z,4.0,cecb7ece-fb70-4a7f-b51d-6e8dd1506ad9,452.0,3d10019f-c88e-3de5-9916-6107b9c0263d,955.0,...,NaN,NaN,NaN,NaN,54.16,4860.0,195662009.0,691.0,Acute viral pharyngitis (disorder),691.0


La feuille décrit la liste des rencontres avec un patient.
Le motif le plus répandu est l'examination d'ordre général, avec près de 30% d'apparition.
La raison la plus répandue n'est pas remplie, et représente environ les trois quarts des rencontres.
L'immense majorité (99%) des rencontres sont facturées à 129,16 dollars, et à peine 25% d'entre elles sont entièrement couvertes.

#### Organizations

In [22]:
scan_report.parse("organizations.csv").head(5)

,Id,Frequency,NAME,Frequency.1,ADDRESS,Frequency.2,CITY,Frequency.3,STATE,Frequency.4,...,LAT,Frequency.6,LON,Frequency.7,PHONE,Frequency.8,REVENUE,Frequency.9,UTILIZATION,Frequency.10
0,db0acede-4abe-3c01-8d03-5c68a190d8c7,1.0,MINUTECLINIC DIAGNOSTIC OF MASSACHUSETTS LLC,13.0,1180 BEACON ST,6.0,BROOKLINE,32.0,MA,1119.0,...,42.331960,34.0,-71.020173,34.0,NaN,184.0,3616.48,70.0,28.0,68.0
1,6f8f02cd-5231-36b6-8174-1cb9cb653c01,1.0,UMASS MEMORIAL MEDICAL GROUP INC.,12.0,955 MAIN ST,5.0,CAMBRIDGE,29.0,NaN,NaN,...,42.323853,32.0,-71.142003,32.0,866-389-2727,13.0,0.00,43.0,30.0,33.0
2,eb41a4ec-fbbe-336c-9987-cae575551845,1.0,MEDICAL CARE OF BOSTON MANAGEMENT CORPORATION,9.0,199 REEDSDALE RD,5.0,BOSTON,25.0,NaN,NaN,...,42.376043,27.0,-71.118680,27.0,800-865-3384,5.0,3874.80,33.0,31.0,31.0
3,1ac92984-69ab-3c07-9b52-73736930a1b5,1.0,SOUTHCOAST PHYSICIANS GROUP INC..,7.0,3 WOODLAND RD,4.0,MILTON,18.0,NaN,NaN,...,42.390445,17.0,-71.101033,17.0,800-838-6331,3.0,4003.96,29.0,29.0,25.0
4,95f1fe9a-3755-3b65-b23f-b6fdb9cc713d,1.0,STEWARD MEDICAL GROUP INC,7.0,1 BROOKLINE PL,3.0,SOMERVILLE,17.0,NaN,NaN,...,42.241557,17.0,-71.082432,17.0,617-912-7900,2.0,3745.64,24.0,34.0,23.0


Le fichier présente la liste des établissements de santé et leurs informations administratives.
On remarque 43 établissements au revenu nul, est-ce une codification pour un revenu négatif ?
Beaucoup de numéros ne sont pas renseignés.
On remarque que 13 organisations qui utilisent le même nom, mais seulement 6 au plus ont la même adresse, alors que 34 ont la même latitude et la même longitude...

### Remarques générales sur les feuilles

L'usage du champ "Frequency", qui groupe les colonnes par valeurs apparaissant de multiples fois pour en sortir la somme, pourrait permettre de savoir quels encodages / mapping développer en priorité si le problème était posé.
Cependant, son utilisation casse la continuité au sein des lignes, et il est alors impossible de vérifier si les lignes sont cohérentes, par exemple :
- La date de début est-elle plus ancienne que la date de fin ?
- Les unités d'une observation sont elles cohérentes avec le type et la valeur de la mesure ?

Nous partirons donc du principe ne contiennent pas d'erreur, à part la colonne "STOP" de la feuille device qui est vide.

Le rapport contient également une feuille "_" qui contient les informations liées à la version de WhiteRabbit, qui peut être utile pour vérifier les conditions de génération du rapport en cas de résultats aberrants dans ce dernier.